## ETL Pipeline for Pre-Processing the Files

---

In [1]:
# Import Python packages
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

from cassandra_queries import *
from create_tables import *

### Data Pre-Processing via CSV Files

---

In [2]:
# Checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list[0:3])

C:\Users\JoseppBenvenuto\Desktop\Projects\Udacity_DE\Python_Cassandra_ETL_Pipeline
['C:\\Users\\JoseppBenvenuto\\Desktop\\Projects\\Udacity_DE\\Python_Cassandra_ETL_Pipeline/event_data\\2018-11-01-events.csv', 'C:\\Users\\JoseppBenvenuto\\Desktop\\Projects\\Udacity_DE\\Python_Cassandra_ETL_Pipeline/event_data\\2018-11-02-events.csv', 'C:\\Users\\JoseppBenvenuto\\Desktop\\Projects\\Udacity_DE\\Python_Cassandra_ETL_Pipeline/event_data\\2018-11-03-events.csv']


In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
 
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline = '') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
         # extracting each data row one by one and append it        
        for line in csvreader:
            # print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print('Number of rows: ' + str(len(full_data_rows_list)))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
print(full_data_rows_list[0:3])

Number of rows: 8056
[['', 'Logged In', 'Walter', 'M', '0', 'Frye', '', 'free', 'San Francisco-Oakland-Hayward, CA', 'GET', 'Home', '1.54092E+12', '38', '', '200', '1.54111E+12', '39'], ['', 'Logged In', 'Kaylee', 'F', '0', 'Summers', '', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'GET', 'Home', '1.54034E+12', '139', '', '200', '1.54111E+12', '8'], ["Des'ree", 'Logged In', 'Kaylee', 'F', '1', 'Summers', '246.30812', 'free', 'Phoenix-Mesa-Scottsdale, AZ', 'PUT', 'NextSong', '1.54034E+12', '139', 'You Gotta Be', '200', '1.54111E+12', '8']]


In [4]:
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
# Creates a csv reader dialect for open function
csv.register_dialect('myDialect', quoting = csv.QUOTE_ALL, skipinitialspace = True)

# Writes new csv with all rows from all csv files
with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline = '') as f:
    # Uses dialect object
    writer = csv.writer(f, dialect = 'myDialect')
    # Writes column headers
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    # for each row write only specific columns selected by the indicated indexes below
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        # Selects columns by index
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

In [5]:
# Check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print('Final number of rows: ' + str(sum(1 for line in f)))

Final number of rows: 6821


### Create Cluster & Keyspace

---

In [6]:
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
    print('Cluster created')

except Exception as e:
    print(e)

Cluster created


In [7]:
# Create a Keyspace 
# For one node code
try:
    session.execute('''CREATE KEYSPACE IF NOT EXISTS sparkify 
                       WITH REPLICATION =
                       {'class':'SimpleStrategy', 
                       'replication_factor':1}''')
    print('Sparkify keyspace created')

except Exception as e:
    print(e)

Sparkify keyspace created


In [8]:
# Connect to keyspace
try:
    session.set_keyspace('sparkify')
    print('Sparkify keyspace set')

except Exception as e:
    print(e)

Sparkify keyspace set


### Create Tables

---

In [9]:
# Create tables
'''
2.3.1  1.
Table for query Give me the artist, song title and song's length in the music app history that 
was heard during sessionId = 338, and itemInSession = 4
'''
try: 
    session.execute(songs_by_session_and_item_create)
    print('songs_by_session_and_item created')

except Exception as e:
    print(e)

songs_by_session_and_item created


In [10]:
'''
2.3.2  2. 
Table to query name of artist, song (sorted by itemInSession) 
and user (first and last name) for userid = 10, sessionid = 182
'''
try: 
    session.execute(songs_by_user_and_session_create)
    print('songs_by_user_and_session created')

except Exception as e:
    print(e)

songs_by_user_and_session created


In [11]:
'''
2.3.3  3. 
Table to query every user name (first and last) in my music app history 
who listened to the song 'All Hands Against His Own'
'''
query = ''''''

try: 
    session.execute(songs_by_user_create)
    print('songs_by_user created')

except Exception as e:
    print(e)

songs_by_user created


### Insert Data

---

In [12]:
# Insert data to sparkify keyspace tables
file = 'event_datafile_new.csv'

try: 
    with open(file, 'r', encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            # Data for artist, song title and song's length in the music app history that 
            # was heard during sessionId = 338, and itemInSession = 4
            session.execute(songs_by_session_and_item_insert,(int(line[8]),int(line[3]),line[0],line[9],float(line[5])))
            # Data for only the following: name of artist, song (sorted by itemInSession) and user 
            # (first and last name) for userid = 10, sessionid = 182
            session.execute(songs_by_user_and_session_insert,(int(line[10]),int(line[8]),int(line[3]),line[0],line[9],line[1],line[4]))
            # Data for every user name (first and last) in my music app history who listened to the 
            # song 'All Hands Against His Own'
            session.execute(songs_by_user_insert,(line[9],int(line[10]),line[1],line[4]))
    print('Inserted data for songs_by_session_and_item, songs_by_user_and_session, songs_by_user')
        
except Exception as e:
    print(e)

Inserted data for songs_by_session_and_item, songs_by_user_and_session, songs_by_user


### Query Data

---

In [13]:
# Query for the artist, song title and song's length in the music app history that was heard during
# sessionId = 338, and itemInSession = 4
query = '''SELECT artist, song, length
           FROM songs_by_session_and_item
           WHERE sessionId = 338 AND itemInSession = 4'''
try: 
    rows = session.execute(query)
    print('Query successful')

except Exception as e:
    print(e)

for row in rows:
    print(row)
#     print(row.artist, row.song, row.length)

Query successful
Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=Decimal('495.3073'))


In [14]:
# Query for the name of artist, song (sorted by itemInSession) and user (first and last name) for 
# userid = 10, sessionid = 182
query = ('''SELECT artist, song, first_name, Last_name
            FROM songs_by_user_and_session
            WHERE userId = 10 AND sessionId = 182
            ORDER BY itemInSession DESC
''')
try: 
    rows = session.execute(query)
    print('Query successful')

except Exception as e:
    print(e)

for row in rows:
    print(row)
#     print(row.artist, row.song, row.first_name, row.last_name)

Query successful
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', first_name='Sylvie', last_name='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', first_name='Sylvie', last_name='Cruz')
Row(artist='Three Drives', song='Greece 2000', first_name='Sylvie', last_name='Cruz')
Row(artist='Down To The Bone', song="Keep On Keepin' On", first_name='Sylvie', last_name='Cruz')


In [15]:
# Query for every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = ('''SELECT first_name, Last_name, song, userId
            FROM songs_by_user
            WHERE song = 'All Hands Against His Own'
''')
try: 
    rows = session.execute(query)
    print('Query successful')

except Exception as e:
    print(e)
    
for row in rows:
    print(row)
#     print(row.first_name, row.last_name)

Query successful
Row(first_name='Jacqueline', last_name='Lynch', song='All Hands Against His Own', userid=29)
Row(first_name='Tegan', last_name='Levine', song='All Hands Against His Own', userid=80)
Row(first_name='Sara', last_name='Johnson', song='All Hands Against His Own', userid=95)


### Drop Tables

---

In [18]:
# Drop all tables in sparkify keyspace
drop_tables(session)

Tables in sparkify keyspace have been dropped


In [25]:
# Query to view tables in sparkify keysapce
try:
    tables = session.execute('''SELECT * FROM system_schema.columns
                                WHERE keyspace_name = 'sparkify'
    ''')
    
except Exception as e:
    print(e)

# Get list of tables
table_list = []

for table in tables:
    table_list.append(table)

# View tables
if len(table_list) > 0:
    for table in table_list:
        print(table)

elif len(table_list) > 0:
    print('There are no tables in sparkify keyspace')

There are no tables in sparkify keyspace


### Close Session and Cluster Connection

---

In [27]:
session.shutdown()
cluster.shutdown()